# Secure Data Disclosure: Client side

This notebook showcases how researcher could use the Secure Data Disclosure system. It explains the different functionnalities provided by the `lomas_client` library to interact with the secure server.

The secure data are never visible by researchers. They can only access to differentially private responses via queries to the server.

Each user has access to one or multiple projects and for each dataset has a limited budget with $\epsilon$ and $\delta$ values.

In [ ]:
from IPython.display import Image
Image(filename="images/image_demo_client.png", width=800)

We will use the Synthetic Swiss Income Dataset to demonstrate the how to use the library `lomas_client` with polars queries.

## Step 1: Install the library

It can be installed via the pip command:

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
# !pip install lomas_client

In [ ]:
from lomas_client import Client
import numpy as np
import opendp.prelude as dp

## Step 2: Initialise the client

Once the library is installed, a Client object must be created. It is responsible for sending sending requests to the server and processing responses in the local environment. It enables a seamless interaction with the server. 

To create the client, we need to give it a few parameters:
- a url: the root application endpoint to the remote secure server.
- user_name: user name as registered in the database
- dataset_name: the name of the dataset that we want to query (FSO_INCOME_SYNTHETIC)

In [ ]:
APP_URL = "http://lomas_server_dev:80"
USER_NAME = "Dr. FSO"
DATASET_NAME = "FSO_INCOME_SYNTHETIC"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

## Step 3: Metadata and dummy dataset

### Getting dataset metadata

Dr. FSO has never seen the data and as a first step to understand what is available to her, she would like to check the metadata of the dataset. Therefore, she just needs to call the `get_dataset_metadata()` function of the client. As this is public information, this does not cost any budget.

This function returns metadata information in a format based on [SmartnoiseSQL dictionary format](https://docs.smartnoise.org/sql/metadata.html#dictionary-format), where among other, there is information about all the available columns, their type, bound values (see Smartnoise page for more details). Any metadata is required for Smartnoise-SQL is also required here and additional information such that the different categories in a string type column column can be added.

In [ ]:
income_metadata = client.get_dataset_metadata()
income_metadata

### Get a dummy dataset

Now, that she has seen and understood the metadata, she wants to get an even better understanding of the dataset (but is still not able to see it). A solution to have an idea of what the dataset looks like it to create a dummy dataset. 

Based on the public metadata of the dataset, a random dataframe can be created created. By default, there will be 100 rows and the seed is set to 42 to ensure reproducibility, but these 2 variables can be changed to obtain different dummy datasets.
Getting a dummy dataset does not affect the budget as there is no differential privacy here. It is not a synthetic dataset and all that could be learn here is already present in the public metadata (it is created randomly on the fly based on the metadata).

Dr. FSO first create a dummy dataset with 200 rows and chooses a seed of 0.

In [ ]:
NB_ROWS = 200
SEED = 0

In [ ]:
dummy_lf = client.get_dummy_lf(nb_rows=NB_ROWS, seed = SEED)

dummy_lf.collect()

In [ ]:
test = client.get_dummy_dataset(nb_rows=NB_ROWS, seed = SEED)

In [ ]:
test.dtypes

## Step 4: Prepare the pipeline

It is necessary to prepare the pipeline before sending the query to the client.

In [ ]:
import polars as pl

### a. mean

In [ ]:
# Income bounds
income_lower_bound, income_upper_bound = income_metadata["columns"]["income"]["lower"], income_metadata["columns"]["income"]["upper"]

In [ ]:
plan = dummy_lf.select(
    pl.col("income").dp.mean(bounds=(income_lower_bound, income_upper_bound), scale=(100,1)
))

## Step 5: Send the queries

### a. Dummy queries

In [ ]:
res = client.opendp.query(plan, dummy = False, nb_rows = NB_ROWS, seed=SEED)

print(res.result.value)

By default, the client will use a laplace mechanism. Note that Dr. FSO can also use a Gaussian mechanism if wanted.

In [ ]:
res = client.opendp.query(plan, mechanism = "gaussian", fixed_delta = 1.0, dummy = True, nb_rows = NB_ROWS, seed=SEED)

print(res.result.value)

### b. budget estimation

In [ ]:
res = client.opendp.cost(plan, mechanism = "laplace")

print(res)

In [ ]:
res = client.opendp.cost(plan, mechanism = "gaussian", fixed_delta = 0.001)

print(res)

### c. Actual queries

In [ ]:
res = client.opendp.query(plan, mechanism = "laplace")

print(res.result.value)

In [ ]:
res = client.opendp.query(plan, mechanism = "gaussian", fixed_delta = 0.001)

print(res.result.value)

## Group_by examples

In [ ]:
# Grouped by "sex"
plan = dummy_lf.group_by("sex").agg([
    pl.col("income").dp.mean(bounds=(income_lower_bound, income_upper_bound), scale=(100.0,1))
])

In [ ]:
res = client.opendp.query(plan, dummy=True)
print(res.result.value)

In [ ]:
plan = dummy_lf.group_by(["sex","region"]).agg([
    pl.col("income").dp.mean(bounds=(income_lower_bound, income_upper_bound), scale=(100.0,1))
])

In [ ]:
res = client.opendp.query(plan, dummy=True)

print(res.result.value)

In [ ]:
res = client.opendp.cost(plan, mechanism = "gaussian", fixed_delta = 0.001)

print(res)

## Quantile examples

In [ ]:
candidates = list(range(1_000, 100_000, 1000))

In [ ]:
plan = dummy_lf.select(
    pl.col("income").dp.quantile(alpha=.75, candidates=candidates, scale=1.)
)

In [ ]:
res = client.opendp.query(plan)

print(res.result.value)

### Quantile with group_by

In [ ]:
plan = dummy_lf.group_by("sex").agg([
    pl.col("income").dp.quantile(alpha=.25, candidates=candidates, scale=(1.)),
])

In [ ]:
res = client.opendp.query(plan)

print(res.result.value)

### Quantile with group_by on mulitple columns

In [ ]:
plan = dummy_lf.group_by(["sex", "region"]).agg([
    pl.col("income").dp.quantile(alpha=.75, candidates=candidates, scale=(1.)),
])

In [ ]:
res = client.opendp.query(plan)

print(res.result.value)

### Income distribution for partitions of the population:
#### Prepare the pipeline

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Partitions
PARTITIONS = ['sex', 'region']
# Prepare a list of candidates
candidates = [x * 250.0 for x in range(8, 52)]

In [ ]:
def make_quantile_pipeline(quantile):
    # Create expression
    return dummy_lf.group_by(["sex", "region"]).agg([
        pl.col("income").dp.quantile(alpha=quantile, candidates=candidates, scale=1.),
    ])

In [ ]:
q25 = make_quantile_pipeline(0.25)
q50 = make_quantile_pipeline(0.5)
q75 = make_quantile_pipeline(0.75)

In [ ]:
r25 = client.opendp.query(q25)
r50 = client.opendp.query(q50)
r75 = client.opendp.query(q75)

Let us put together the results and show them in a table. Notice that the output is a polars dataframe, we thus need to transform it to a pandas DataFrame if we want to work with pandas.

In [ ]:
r25 = r25.result.value.to_pandas()
r50 = r50.result.value.to_pandas()
r75 = r75.result.value.to_pandas()

In [ ]:
results = pd.merge(r25, r50, on=PARTITIONS, suffixes=('_25', '_50'))
results = pd.merge(results, r75, on=PARTITIONS)
results.sort_values(by = ['region', 'sex']).head()

#### Visualise results

In [ ]:
def quantile_data(q1, q2, q3):
    return np.concatenate((np.random.uniform(q1, q2, size=50), np.random.uniform(q2, q3, size=50)))

results['data'] = results.apply(
    lambda row: quantile_data(row["income_25"], row["income_50"], row["income"]),
    axis=1,
)
results['sex'] = results['sex'].replace({0: 'woman', 1: 'man'})
results['region'] = results['region'].replace({1: 'Lemanique', 2: 'Mittleland', 3: 'North-West', 4: 'Zürich', 5: 'Oriental', 6: 'Central', 7: 'Ticino'})
results = results.explode('data', ignore_index=True)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="region", y="data", hue="sex", data=results, palette="Set1", width=0.5);
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Regions', fontsize=15)
plt.ylabel('Income per month (in CHF)', fontsize=15)
plt.title('Income per partition of the population', fontsize=16)
plt.show()